<a href="https://colab.research.google.com/github/Paul-Richmond/hepthLlama/blob/main/vLLM_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q vllm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.3/228.3 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6

In [ ]:
!pip install bitsandbytes>=0.44.0

In [ ]:
import huggingface_hub
import os
import re
from vllm import LLM, SamplingParams
from vllm.lora.request import LoRARequest
from datasets import load_dataset

In [ ]:
huggingface_hub.login()

In [ ]:
dataset_name = 'LLMsForHepth/hep-th_primary'
base_model_name = 'meta-llama/Meta-Llama-3.1-8B'
lora_adapter_name = 'LLMsForHepth/s1-L-3.1-8B-base'

generation_cfg = {'max_tokens': 1024,
                  'min_tokens': 48}

model_cfg = {'dtype': 'half',
             'quantization': "bitsandbytes",
             'load_format': "bitsandbytes",
             'max_model_len': 1024,
             'gpu_memory_utilization': 0.6}

In [ ]:
def split_abstracts(example):
    """
    Splits an abstract into a prompt and ground truth.

    The prompt is created from the first half (or slightly more) of the sentences in the abstract,
    and the ground truth is the remaining sentences.
    If there is only a single sentence then we split instead on spaces to get, roughly, words
    and take the first half (or slightly more) of these.

    Args:
        example (dict): A dictionary containing the 'abstract' text to be split.

    Returns:
        dict: A dictionary with 'prompt' and 'y_true' keys containing the split abstract parts.
    """
    text = example['abstract']
    # Split the abstract into sentences (i.e. text sequences which end with any of .!? and a space)
    sentences = re.split(r'(?<=[.!?])\s+', text)
    # Calculate the split point
    total_sentences = len(sentences)
    if total_sentences > 1:  # more than 1 sentence so can split
        split_point = (total_sentences + 1) // 2  # Ensures the prompt has >= number of sentences than y_true
        # Join the sentences back into two parts
        prompt = ' '.join(sentences[:split_point])
        y_true = ' '.join(sentences[split_point:])
    else:  # only a single sentence so split on words (latex commands between $$ might get split)
        words = text.split()
        total_words = len(words)
        split_point = (total_words + 1) // 2  # Ensures the prompt has >= number of sentences than y_true
        # Join the sentences back into two parts
        prompt = ' '.join(words[:split_point])
        y_true = ' '.join(words[split_point:])
    return {'prompt': prompt, 'y_true': y_true}

In [ ]:
ds = load_dataset(dataset_name, split='test')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.72k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/80.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/73768 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/15808 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/15808 [00:00<?, ? examples/s]

In [ ]:
ds = ds.remove_columns(list(set(ds.column_names) - {'id', 'abstract'}))
ds = ds.map(split_abstracts,
            batched=False,
            desc='Splitting abstracts')

Splitting abstracts:   0%|          | 0/15808 [00:00<?, ? examples/s]

In [ ]:
sampling_params = SamplingParams(**generation_cfg)
# sampling_params = SamplingParams()

In [ ]:
lora_request = LoRARequest("sql_adapter", 1, lora_path=lora_adapter_name)

<ipython-input-11-189c68ce2c41>:1: DeprecationWarning: The 'lora_local_path' attribute is deprecated and will be removed in a future version. Please use 'lora_path' instead.
  lora_request = LoRARequest("sql_adapter", 1, lora_path=lora_adapter_name)


In [ ]:
llm = LLM(model=base_model_name, enable_lora=True, **model_cfg)

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

WARNING 10-03 14:58:38 config.py:1656] Casting torch.bfloat16 to torch.float16.
WARNING 10-03 14:58:38 config.py:319] bitsandbytes quantization is not fully optimized yet. The speed can be slower than non-quantized models.
WARNING 10-03 14:58:38 config.py:1552] bitsandbytes quantization is not tested with LoRA yet.
INFO 10-03 14:58:38 llm_engine.py:226] Initializing an LLM engine (v0.6.1.dev238+ge2c6e0a82) with config: model='meta-llama/Meta-Llama-3.1-8B', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=1024, download_dir=None, load_format=LoadFormat.BITSANDBYTES, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=bitsandbytes, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cud

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 10-03 15:01:48 model_runner.py:1025] Loading model weights took 5.3460 GB
INFO 10-03 15:02:08 gpu_executor.py:122] # GPU blocks: 541, # CPU blocks: 2048
INFO 10-03 15:02:11 model_runner.py:1329] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 10-03 15:02:11 model_runner.py:1333] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 10-03 15:03:52 model_runner.py:1456] Graph capturing finished in 102 secs.


In [ ]:
prompts = ds['prompt'][:10] # slice for smaller list

In [ ]:
outputs = llm.generate(prompts, sampling_params)

Processed prompts: 100%|██████████| 10/10 [02:09<00:00, 12.97s/it, est. speed input: 8.18 toks/s, output: 9.79 toks/s]


In [ ]:
completions = []
for output in outputs:
    completions.append(output.outputs[0].text)

In [ ]:
prompts[0]

'We demonstrate that exceptional field theory is a truncation of the non-linear realisation of the semi-direct product of E11 and its first fundamental as proposed in 2003. Evaluating the simple equations of the E11 approach, and using the commutators of the E11 algebra, we find the equations of exceptional field theory after making a radical truncation.'

In [ ]:
prompts[0] + completions[0]

'We demonstrate that exceptional field theory is a truncation of the non-linear realisation of the semi-direct product of E11 and its first fundamental as proposed in 2003. Evaluating the simple equations of the E11 approach, and using the commutators of the E11 algebra, we find the equations of exceptional field theory after making a radical truncation. It emerges that the massless equations in exceptional field theory take a surprisingly simple form when expressed in a particular 23 dimensional vector ( potential in this context) and a 12 dimensional rank-two matrix ( generically referred to as t in exceptional field theory). We also explore a proposal for the action underlying the theory and check that known anomalous symmetries are artefacts of E11-adapted formulae in E8 X SU(8).'

In [ ]:
ds['abstract'][0]

'We demonstrate that exceptional field theory is a truncation of the non-linear realisation of the semi-direct product of E11 and its first fundamental as proposed in 2003. Evaluating the simple equations of the E11 approach, and using the commutators of the E11 algebra, we find the equations of exceptional field theory after making a radical truncation. This procedure does not respect any of the higher level E11 symmetries and so these are lost. We suggest that the need for the section condition in exceptional field theory could be a consequence of the truncation.'